<a href="https://colab.research.google.com/github/Mityukovanna/pet_project_voice_assistant/blob/main/BERT_fine_tuning_for_voice_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the environment

In [ ]:
! pip install -q huggingface_hub

# Loading dataset

In [ ]:
!pip install -q transformers datasets

In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [ ]:
!pip install -q transformers datasets

In [ ]:
from datasets import *

In [ ]:
# loading unsorted dataset from my personal account on Hugging Face
unsorted_ds = load_dataset('Amityukova/pet_project_voice_assistant_dataset')

# shuffling dataset
ds = unsorted_ds.shuffle(seed=42)
# separating dataset into test, evaluate and train
train_testvalid = ds['train'].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'greeting', 'farewell', 'current weather', 'weather forecast', 'time', 'turn on music', 'look in wikipedia', 'joke'],
        num_rows: 2515
    })
    test: Dataset({
        features: ['prompt', 'greeting', 'farewell', 'current weather', 'weather forecast', 'time', 'turn on music', 'look in wikipedia', 'joke'],
        num_rows: 315
    })
    validation: Dataset({
        features: ['prompt', 'greeting', 'farewell', 'current weather', 'weather forecast', 'time', 'turn on music', 'look in wikipedia', 'joke'],
        num_rows: 314
    })
})


creating a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [ ]:
labels = [label for label in dataset['train'].features.keys() if label not in ['prompt']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [ ]:
labels

['greeting',
 'farewell',
 'current weather',
 'weather forecast',
 'time',
 'turn on music',
 'look in wikipedia',
 'joke']

# Pre-processing

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["prompt"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding


In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/2515 [00:00<?, ? examples/s]

Map:   0%|          | 0/315 [00:00<?, ? examples/s]

Map:   0%|          | 0/314 [00:00<?, ? examples/s]

Checking how the code works on example. This is needed to ensure that everything working as it should


In [ ]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['prompt', 'greeting', 'farewell', 'current weather', 'weather forecast', 'time', 'turn on music', 'look in wikipedia', 'joke', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [ ]:
tokenizer.decode(example['input_ids'])

'[CLS] tell me a joke that ’ s sure to make me laugh out loud. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
example['labels']

[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['turn on music']

In [ ]:
# formatting dataset to pytorch tesnors
encoded_dataset.set_format("torch")

# Defining model


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training the model

In [ ]:
!pip install -q accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 31.0 MB/s eta 0:00:00


In [ ]:
batch_size = 8
metric_name = "f1"

**The instructions to run the args correctly:**

Run pip install accelerate -U in a cell

In the top menu click Runtime → Restart Runtime

Do not rerun any cells with !pip install in them Rerun all the other code cells and you should be good to go!

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"BERT_trained_for_voice_assistant",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Computing metrics of the model

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result


verifying a batch as well as the forward tensor

In [ ]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [ ]:
encoded_dataset['train']['input_ids'][0]

tensor([ 101, 2425, 2033, 1037, 8257, 2008, 1521, 1055, 2469, 2000, 2191, 2033,
        4756, 2041, 5189, 1012,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.5401, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0680, -0.0433, -0.2012, -0.6383, -0.4389, -0.4234, -0.3632,  0.8023]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Training the model

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.062911,0.939189,0.942675,0.885350
2,0.129000,0.036668,0.970874,0.976797,0.955414
3,0.129000,0.023539,0.990446,0.994540,0.987261
4,0.031700,0.018525,0.993631,0.996360,0.993631
5,0.019200,0.017648,0.993631,0.996360,0.993631


TrainOutput(global_step=1575, training_loss=0.057857502225845577, metrics={'train_runtime': 18001.7878, 'train_samples_per_second': 0.699, 'train_steps_per_second': 0.087, 'total_flos': 827199940454400.0, 'train_loss': 0.057857502225845577, 'epoch': 5.0})

In [ ]:
while True:
  pass

KeyboardInterrupt: 

# Evaluating the model

In [ ]:
trainer.evaluate()

{'eval_loss': 0.01852518506348133,
 'eval_f1': 0.9936305732484076,
 'eval_roc_auc': 0.9963603275705186,
 'eval_accuracy': 0.9936305732484076,
 'eval_runtime': 121.932,
 'eval_samples_per_second': 2.575,
 'eval_steps_per_second': 0.328,
 'epoch': 5.0}

# Inference

In [ ]:
text = "who is mr beast"
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [ ]:
logits = outputs.logits
logits.shape

torch.Size([1, 8])

In [ ]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['look in wikipedia']


# Saving model

In [ ]:
trained_model = trainer
trained_model.push_to_hub("Amityukova/BERT_finetuned_for_voice_assistant/tree/main")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1719561682.e4447edf2db0.5573.0:   0%|          | 0.00/8.38k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1719579815.e4447edf2db0.5573.1:   0%|          | 0.00/508 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Amityukova/BERT_trained_for_voice_assistant/commit/53f49970ed2f9f7cf9baf1305ed0f9afd9181f82', commit_message='Amityukova/BERT_finetuned_for_voice_assistant/tree/main', commit_description='', oid='53f49970ed2f9f7cf9baf1305ed0f9afd9181f82', pr_url=None, pr_revision=None, pr_num=None)